# 10. Apply for a Loan

Now that Alice has a job, she’d like to apply for a loan. That will require a proof of employment.
She can get this from the **Job-Certificate** credential offered by Acme.
Alice goes through a familiar sequence of interactions.

1. First she creates a Credential Request.
 ```python  
  # Alice Agent
    (job_certificate_cred_request_json, job_certificate_cred_request_metadata_json) = \
        await anoncreds.prover_create_credential_req(alice_wallet, alice_acme_did, job_certificate_cred_offer_json,
                                                     acme_job_certificate_cred_def, alice_master_secret_id)
 ```

 2. Acme issues a **Job-Certificate** Credential for Alice.
 ```python
  # Acme Agent
  alice_job_certificate_cred_values_json = json.dumps({
      "first_name": {"raw": "Alice", "encoded": "245712572474217942457235975012103335"},
      "last_name": {"raw": "Garcia", "encoded": "312643218496194691632153761283356127"},
      "employee_status": {"raw": "Permanent", "encoded": "2143135425425143112321314321"},
      "salary": {"raw": "2400", "encoded": "2400"},
      "experience": {"raw": "10", "encoded": "10"}
  })
  job_certificate_cred_json, _, _ = \
      await anoncreds.issuer_create_credential(acme_wallet, job_certificate_cred_offer_json,job_certificate_cred_request_json,
                                               alice_job_certificate_cred_values_json, None, None)
```

Now the **Job-Certificate** Credential has been issued and Alice now has it in her possession. Alice stores **Job-Certificate** Credential in her wallet.
```python
  # Alice Agent
  await anoncreds.prover_store_credential(alice_wallet, None, job_certificate_cred_request_json, job_certificate_cred_request_metadata_json,
                                          job_certificate_cred_request_json, acme_job_certificate_cred_def_json, None)
```

She can use it when she applies for her loan, in much the same way that she used her transcript when applying for a job.

There is a disadvantage in this approach to data sharing though, — it may disclose more data than what is strictly necessary. If all Alice needs to do is provide proof of employment, this can be done with an anonymous credential instead. Anonymous credentials may prove certain predicates without disclosing actual values (e.g., Alice is employed full-time, with a salary greater than X, along with her hire date, but her actually salary remains hidden). A compound proof can be created, drawing from credentials from both Faber College and Acme Corp, that discloses only what is necessary.

Alice now establishes connection with Thrift Bank.

Alice gets a **Loan-Application-Basic** Proof Request from Thrift Bank that looks like:
```python
  # Thrift Agent
  apply_loan_proof_request_json = json.dumps({
      'nonce': '123432421212',
      'name': 'Loan-Application-Basic',
      'version': '0.1',
      'requested_attributes': {
          'attr1_referent': {
              'name': 'employee_status',
              'restrictions': [{'cred_def_id': acme_job_certificate_cred_def_id}]
          }
      },
      'requested_predicates': {
          'predicate1_referent': {
              'name': 'salary',
              'p_type': '>=',
              'p_value': 2000,
              'restrictions': [{'cred_def_id': acme_job_certificate_cred_def_id}]
          },
          'predicate2_referent': {
              'name': 'experience',
              'p_type': '>=',
              'p_value': 1,
              'restrictions': [{'cred_def_id': acme_job_certificate_cred_def_id}]
          }
      }
  })
```

Alice has only one credential that meets the proof requirements for this **Loan-Application-Basic** Proof Request.
```python
  # Alice Agent
  {
      'referent': 'Job-Certificate Credential Referent',
      'revoc_reg_seq_no': None,
      'schema_id': job_certificate_schema_id,
      'cred_def_id': acme_job_certificate_cred_def_id,
      'attrs': {
          'employee_status': 'Permanent',
          'last_name': 'Garcia',
          'experience': '10',
          'first_name': 'Alice',
           'salary': '2400'
      }
  }
```

For the **Loan-Application-Basic** Proof Request Alice divided attributes as follows:
```python
  # Alice Agent
  apply_loan_requested_creds_json = json.dumps({
      'self_attested_attributes': {},
      'requested_attributes': {
          'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True}
      },
      'requested_predicates': {
          'predicate1_referent': {'cred_id': cred_for_predicate1['referent']},
          'predicate2_referent': {'cred_id': cred_for_predicate2['referent']}
      }
  })
```

Alice creates the Proof for the **Loan-Application-Basic** Proof Request.
```python
  # Alice Agent
  alice_apply_loan_proof_json = \
      await anoncreds.prover_create_proof(alice_wallet, authdecrypted_apply_loan_proof_request_json, apply_loan_requested_creds_json,
                                          alice_master_secret_id, schemas_json, cred_defs_json, revoc_states_json)
```

Alice sends just the **Loan-Application-Basic** proof to the bank.
This allows her to minimize the PII (personally identifiable information) that she has to share when all she's trying to do right now is prove basic eligibility.

When **Thrift** inspects the received Proof he will see following structure:
```
  # Thrift Agent
  {
      'requested_proof': {
          'revealed_attributess': {
              'attr1_referent': {'sub_proof_index': 0, 'raw':'Permanent', 'encoded':'2143135425425143112321314321'},
          },
          'self_attested_attrs': {},
          'unrevealed_attrs': {},
          'predicates': {
              'predicate1_referent': {'sub_proof_index': 0},
              'predicate2_referent': {'sub_proof_index': 0}
          }
      },
      'proof' : [] # Validity Proof that Thrift can check
      'identifiers' : [ # Identifiers of credentials were used for Proof building
          'schema_id': job_certificate_schema_id,
          'cred_def_id': acme_job_certificate_cred_def_id,
          'revoc_reg_seq_no': None,
          'timestamp': None
      ]
  }
```

**Thrift Bank** successfully verified the **Loan-Application-Basic** Proof from Alice.
```python
  # Thrift Agent
  assert await anoncreds.verifier_verify_proof(apply_loan_proof_request_json, alice_apply_loan_proof_json,
                                               schemas_json, cred_defs_json, revoc_defs_json, revoc_regs_json)
```

Thrift Bank sends the second Proof Request where Alice needs to share her personal information with the bank.
```python
  # Thrift Agent
  apply_loan_kyc_proof_request_json = json.dumps({
      'nonce': '123432421212',
      'name': 'Loan-Application-KYC',
      'version': '0.1',
      'requested_attributes': {
          'attr1_referent': {'name': 'first_name'},
          'attr2_referent': {'name': 'last_name'},
          'attr3_referent': {'name': 'ssn'}
      },
      'requested_predicates': {}
  })
```

Alice has two credentials that meets the proof requirements for this **Loan-Application-KYC** Proof Request.
```python
  # Alice Agent
  {
    'referent': 'Transcript Credential Referent',
    'schema_id': transcript_schema_id,
    'cred_def_id': faber_transcript_cred_def_id,
    'attrs': {
        'first_name': 'Alice',
        'last_name': 'Garcia',
        'status': 'graduated',
        'degree': 'Bachelor of Science, Marketing',
        'ssn': '123-45-6789',
        'year': '2015',
        'average': '5'
    },
    'rev_reg_id': None,
    'cred_rev_id': None
  },
  {
      'referent': 'Job-Certificate Credentail Referent',
      'schema_key': job_certificate_schema_id,
      'cred_def_id': acme_job_certificate_cred_def_id,
      'attrs': {
          'employee_status': 'Permanent',
          'last_name': 'Garcia',
          'experience': '10',
          'first_name': 'Alice',
          'salary': '2400'
      },
      'rev_reg_id': None,
      'revoc_reg_seq_no': None
  }
```

For the **Loan-Application-KYC** Proof Request Alice divided attributes as follows:
```python
  # Alice Agent
  apply_loan_kyc_requested_creds_json = json.dumps({
      'self_attested_attributes': {},
      'requested_attributes': {
          'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True},
          'attr2_referent': {'cred_id': cred_for_attr2['referent'], 'revealed': True},
          'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True}
      },
      'requested_predicates': {}
  })
```

Alice creates the Proof for **Loan-Application-KYC** Proof Request.
```python
  # Alice Agent
  alice_apply_loan_kyc_proof_json = \
      await anoncreds.prover_create_proof(alice_wallet, apply_loan_kyc_proof_request_json, apply_loan_kyc_requested_creds_json,
                                          alice_master_secret_id, schemas_json, cred_defs_json, revoc_states_json)
```

When **Thrift** inspects the received Proof he will see following structure:
```
  # Thrift Agent
  {
      'requested_proof': {
          'revealed_attributes': {
              'attr1_referent': {'sub_proof_index': 0, 'raw':'123-45-6789', 'encoded':'3124141231422543541'},
              'attr1_referent': {'sub_proof_index': 1, 'raw':'Alice', 'encoded':'245712572474217942457235975012103335'},
              'attr1_referent': {'sub_proof_index': 1, 'raw':'Garcia', 'encoded':'312643218496194691632153761283356127'},
          },
          'self_attested_attrs': {},
          'unrevealed_attrs': {},
          'predicates': {}
      },
      'proof' : [] # Validity Proof that Thrift can check
      'identifiers' : [ # Identifiers of credentials were used for Proof building
          {
            'schema_id': transcript_schema_id,
            'cred_def_id': faber_transcript_cred_def_id,
            'rev_reg_id': None,
            'timestamp': None
          },
          {
            'schema_key': job_certificate_schema_id,
            'cred_def_id': acme_job_certificate_cred_def_id,
            'rev_reg_id': None,
            'timestamp': None
          }
      ]
  }
```

**Thrift Bank** has successfully validated the **Loan-Application-KYC** Proof from Alice.
```python
  # Thrift Agent
  assert await anoncreds.verifier_verify_proof(apply_loan_kyc_proof_request_json, alice_apply_loan_kyc_proof_json,
                                               schemas_json, cred_defs_json, revoc_defs_json, revoc_regs_json)
```

Both of Alice's Proofs have been successfully verified and she got loan from **Thrift Bank**.


[previous](09-alice-applies-for-a-job.ipynb) | [next]()